# ADLI Process Assessment with Radar Charts

This notebook demonstrates ADLI (Approach-Deployment-Learning-Integration) process assessment with dimensional radar chart visualizations.

## Learning Objectives
1. Load and analyze process assessment data
2. Create ADLI radar charts for dimensional profiling
3. Compare ADLI profiles across departments
4. Identify dimensional strengths and weaknesses

In [ ]:
# Import libraries
import sys
sys.path.append('../src')

from adli_letci_core import ADLIIndicators, compute_adli_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries loaded successfully!")

## Load Sample Assessment Data

In [ ]:
# Load sample data
df = pd.read_csv('../data/examples/sample_assessment_data.csv')

# Filter process items only
process_df = df[df['item_type'] == 'Process'].copy()

print(f"Loaded {len(process_df)} process assessments")
print(f"Departments: {process_df['department'].unique()}")
print(f"Categories: {process_df['category'].unique()}")

# Display sample
process_df.head()

## ADLI Radar Chart Function

In [ ]:
def create_adli_radar(data_dict, title="ADLI Profile", ax=None):
    """
    Create ADLI radar chart.
    
    Args:
        data_dict: Dictionary with keys 'Approach', 'Deployment', 'Learning', 'Integration'
        title: Chart title
        ax: Matplotlib axis (if None, creates new figure)
    """
    categories = ['Approach', 'Deployment', 'Learning', 'Integration']
    values = [data_dict[cat] for cat in categories]
    
    # Close the plot by repeating first value
    values += values[:1]
    
    # Compute angle for each axis
    angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
    angles += angles[:1]
    
    # Create plot
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))
    
    # Draw the plot
    ax.plot(angles, values, 'o-', linewidth=2, color='#1f77b4', label='ADLI Profile')
    ax.fill(angles, values, alpha=0.25, color='#1f77b4')
    
    # Set labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=12)
    
    # Set y-axis limits
    ax.set_ylim(0, 1.0)
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=10)
    
    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Title
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    
    return ax

## Example 1: Single Item ADLI Profile

In [ ]:
# Analyze Leadership item 1.1 for Computer Science
item = process_df[(process_df['item_id'] == '1.1') & (process_df['department'] == 'Computer Science')].iloc[0]

adli_data = {
    'Approach': item['approach'],
    'Deployment': item['deployment'],
    'Learning': item['learning'],
    'Integration': item['integration']
}

# Compute score
indicators = ADLIIndicators(**adli_data)
score = compute_adli_score(indicators)

# Create radar chart
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))
create_adli_radar(adli_data, title=f"Item 1.1 - Computer Science\nADLI Score: {score:.2f}/100", ax=ax)
plt.tight_layout()
plt.show()

# Analysis
print("Dimensional Analysis:")
for dim, value in adli_data.items():
    print(f"  {dim:12}: {value:.3f} ({value*100:.1f}%)")
print(f"\nOverall Score: {score:.2f}/100")

## Example 2: Category-Level ADLI Profiles

In [ ]:
# Compute mean ADLI for each category (Computer Science)
cs_df = process_df[process_df['department'] == 'Computer Science']

category_adli = cs_df.groupby('category')[['approach', 'deployment', 'learning', 'integration']].mean()

# Create radar charts for each category
categories = category_adli.index.tolist()
n_cats = len(categories)
n_cols = 3
n_rows = (n_cats + n_cols - 1) // n_cols

fig = plt.figure(figsize=(15, n_rows * 5))

for idx, category in enumerate(categories, 1):
    ax = fig.add_subplot(n_rows, n_cols, idx, projection='polar')
    
    cat_data = {
        'Approach': category_adli.loc[category, 'approach'],
        'Deployment': category_adli.loc[category, 'deployment'],
        'Learning': category_adli.loc[category, 'learning'],
        'Integration': category_adli.loc[category, 'integration']
    }
    
    create_adli_radar(cat_data, title=f"{category}", ax=ax)

plt.suptitle('Computer Science - ADLI Profiles by Category', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## Example 3: Department Comparison Radar Chart

In [ ]:
def create_multi_department_radar(df_by_dept, departments, title="Department Comparison"):
    """
    Create overlay radar chart comparing multiple departments.
    """
    categories = ['Approach', 'Deployment', 'Learning', 'Integration']
    angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    for idx, dept in enumerate(departments):
        dept_data = df_by_dept[df_by_dept['department'] == dept][['approach', 'deployment', 'learning', 'integration']].mean()
        values = dept_data.tolist()
        values += values[:1]
        
        ax.plot(angles, values, 'o-', linewidth=2, color=colors[idx % len(colors)], label=dept)
        ax.fill(angles, values, alpha=0.15, color=colors[idx % len(colors)])
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=12)
    ax.set_ylim(0, 1.0)
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    
    return ax

# Compare three departments
departments = ['Computer Science', 'Business Admin', 'Engineering']
create_multi_department_radar(process_df, departments, title="ADLI Comparison Across Departments")
plt.tight_layout()
plt.show()

## Example 4: Dimensional Strength Analysis

In [ ]:
# Analyze dimensional strengths across all departments
dimension_analysis = process_df.groupby('department')[['approach', 'deployment', 'learning', 'integration']].mean()

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(dimension_analysis.T, annot=True, fmt='.3f', cmap='RdYlGn', 
            vmin=0.5, vmax=1.0, cbar_kws={'label': 'ADLI Score'}, ax=ax)
ax.set_xlabel('Department', fontsize=12)
ax.set_ylabel('ADLI Dimension', fontsize=12)
ax.set_title('ADLI Dimensional Heatmap by Department', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print analysis
print("\nDimensional Strengths and Weaknesses:\n")
for dept in dimension_analysis.index:
    print(f"{dept}:")
    dept_scores = dimension_analysis.loc[dept]
    strongest = dept_scores.idxmax()
    weakest = dept_scores.idxmin()
    print(f"  Strongest: {strongest.capitalize()} ({dept_scores[strongest]:.3f})")
    print(f"  Weakest:   {weakest.capitalize()} ({dept_scores[weakest]:.3f})")
    print()

## Example 5: ADLI Consistency Analysis

In [ ]:
# Compute coefficient of variation for each department
def compute_adli_consistency(df):
    """
    Compute ADLI dimensional consistency (lower CV = more balanced).
    """
    dimensions = ['approach', 'deployment', 'learning', 'integration']
    consistency = []
    
    for dept in df['department'].unique():
        dept_df = df[df['department'] == dept]
        means = dept_df[dimensions].mean()
        cv = means.std() / means.mean()  # Coefficient of variation
        consistency.append({
            'department': dept,
            'cv': cv,
            'mean_adli': means.mean()
        })
    
    return pd.DataFrame(consistency)

consistency_df = compute_adli_consistency(process_df)
consistency_df = consistency_df.sort_values('cv')

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# CV bar chart
ax1.barh(consistency_df['department'], consistency_df['cv'], color='#1f77b4')
ax1.set_xlabel('Coefficient of Variation (CV)', fontsize=12)
ax1.set_ylabel('Department', fontsize=12)
ax1.set_title('ADLI Dimensional Balance\n(Lower CV = More Balanced)', fontsize=13, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Scatter: Balance vs Performance
ax2.scatter(consistency_df['cv'], consistency_df['mean_adli'], s=150, alpha=0.6, edgecolors='black')
for idx, row in consistency_df.iterrows():
    ax2.annotate(row['department'], (row['cv'], row['mean_adli']), 
                fontsize=9, ha='center', va='bottom')
ax2.set_xlabel('Coefficient of Variation', fontsize=12)
ax2.set_ylabel('Mean ADLI Score', fontsize=12)
ax2.set_title('Balance vs Performance', fontsize=13, fontweight='bold')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nADLI Dimensional Balance Analysis:")
print(consistency_df.to_string(index=False))

## Key Insights

### ADLI Radar Charts Reveal:
1. **Dimensional Imbalances**: Identify whether departments have balanced ADLI profiles or specific weaknesses
2. **Comparative Positioning**: Compare departments' process maturity across four dimensions
3. **Improvement Targets**: Visually identify which dimension requires priority attention
4. **Integration Gaps**: Low integration scores indicate siloed operations

### Typical Patterns:
- **Strong Approach, Weak Deployment**: Good methods but limited organizational spread
- **High Deployment, Low Learning**: Widespread but stagnant processes
- **Low Integration**: Category operates independently, lacking strategic alignment

## Next Steps
- **Notebook 03**: LeTCI Results analysis with trend visualization
- **Notebook 05**: Gap prioritization for targeted ADLI improvements